<a href="https://colab.research.google.com/github/Liping-LZ/BDAO_2223/blob/main/Text%20mining/product_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving amazon_products.xlsx to amazon_products.xlsx


In [31]:
import pandas as pd
df = pd.read_excel('amazon_products.xlsx')
df.head()

,product_id,product_name,category,about_product
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,High Compatibility : Compatible With iPhone 12...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,"Compatible with all Type C enabled devices, be..."
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,ã€ Fast Charger& Data Syncã€‘-With built-in s...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,The boAt Deuce USB 300 2 in 1 cable is compati...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,[CHARGE & SYNC FUNCTION]- This cable comes wit...


In [3]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 13.4 MB/s eta 0:00:00


In [32]:
import re
import contractions
import string

df['about_product'] = df.about_product.str.lower()


df['about_product'] = df['about_product'].map(lambda x: contractions.fix(x))


# Remove new line characters ("/n")
df['about_product'] = df['about_product'].map(lambda x: re.sub('\s+', ' ', x))

# Remove url link
df['about_product'] = df['about_product'].apply(lambda x: re.sub('http://\S+|https://\S+', '', x))

# Remove numbers
df['about_product'] = df['about_product'].map(lambda x: re.sub(r"\d+", ' ', x))

# Remove overspace
df['about_product'] = df['about_product'].map(lambda x: re.sub('\s{2,}', " ", x))

# Remove punctuation
df['about_product'] = df['about_product'].map(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

# Remove special characters
df['about_product'] = df['about_product'].map(lambda x: re.sub('\W+',' ', x))

In [33]:
import nltk
nltk.download('punkt')

# create a list of the products from the 'product' column
words = df['about_product'].tolist()

# tokenise the words
from nltk.tokenize import word_tokenize

word_tokens = [] # create an empty list to put the tokenised words
for product in words:
    word_tokens.append(word_tokenize(product))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# Start dealing with stopwords
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
# use English stopwords
stopwords = stopwords.words("english")
# you can also extend the stopwords list with more words that you want to remove
stopwords.extend(['re'])

# create a new list with stop words removed
tokens_without_stopwords = []
for product in word_tokens:
    tokens_without_stopwords.append([w for w in product if not w in stopwords])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# The lemmatizer requires the correct POS tag to be accurate, if you use the default settings of the WordNetLemmatizer.lemmatize(), the default tag is noun.
# Which means the words other than noun would not be lemmatised, and this is not what we expect.
# Therefore we need to define the POS tags ('n' for nouns(default), 'a' for adj, 'v' for verb, 'r' for adverb) which are allowed to be lemmatised.

lemmatised = []
for product in tokens_without_stopwords: 
  n_lemmatised = [lemmatizer.lemmatize(w) for w in product]
  v_lemmatised = [lemmatizer.lemmatize(w,'v') for w in n_lemmatised]
  r_lemmatised = [lemmatizer.lemmatize(w,'r') for w in v_lemmatised]
  a_lemmatised = [lemmatizer.lemmatize(w,'a') for w in r_lemmatised]
  lemmatised.append(a_lemmatised) # here noun, adj, verb, adv are the POS tags that we allowed

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [36]:
# put the tokens back together as text after stemming
import string
rejoin = []
for product in lemmatised: 
    x = " ".join(product) # join the text back together 
    # remove punctuation from the reviews using the string package
    rejoin.append(x)

# add the reformed text to the data frame    
df['filtered_product_description'] = rejoin

In [37]:
df['filtered_product_description'].head()

0    high compatibility compatible iphone x xsmax x...
1    compatible type c enable device android smartp...
2    ã fast charger data syncã build safety proctec...
3    boat deuce usb cable compatible smartphones ta...
4    charge sync function cable come charge data sy...
Name: filtered_product_description, dtype: object

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

# set vectorizer - CountVectorizer for word counts
tfidfvectorizer = TfidfVectorizer()

tfidf = tfidfvectorizer.fit_transform(df.filtered_product_description).toarray()
vocab = tfidfvectorizer.vocabulary_
reverse_vocab = {v:k for k,v in vocab.items()}

feature_names = tfidfvectorizer.get_feature_names()
df_tfidf = pd.DataFrame(tfidf, columns = feature_names)

idx = tfidf.argsort(axis=1)

tfidf_max10 = idx[:,-10:]

df_tfidf['top10'] = [[reverse_vocab.get(item) for item in row] for row in tfidf_max10 ]

tfidf_top = df.join(df_tfidf['top10'], how='left')
# write the dataframe out to csv and download
tfidf_top.to_csv('tfidf_top.csv',index=False)
files.download('tfidf_top.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>